# Import, Load, Settings

In [1]:
import solara
from ipyleaflet import Map, DrawControl, basemaps, Marker, Icon
from ipywidgets import Output, Dropdown, FloatSlider, Text
from IPython.display import display
import json
from pathlib import Path
import datetime as dt
import os
import tomli_w

from DT_flood.utils.workflow_utils import run_scenario, create_workflow_config, run_fa_scenario_workflow

import logging
logging.getLogger('reacton').disabled = True

# vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
# for visualisation, plots
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import colors
from hydromt_sfincs import SfincsModel
from hydromt.data_catalog import DataCatalog
from hydromt_wflow import WflowModel
from hydromt_fiat.fiat import FiatModel
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
proj = ccrs.PlateCarree()
from DT_flood.utils import plot_utils
import threading 
import geopandas as gpd
matplotlib.use('Agg')
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [2]:
# HERE: DROP-DOWN MENU; people won't have datasets on their local drive

FA_databases = ["Scheveningen", "Option B", "Option C"] #################################
FA_database = solara.reactive("Scheveningen")


@solara.component
def Page():
    solara.Select(label="FloodAdapt data base", value=FA_database, values=FA_databases)
    
    solara.Markdown(f"**Selected**: {FA_database.value}")


########################################################################################################################################################################################
# The following line is required only when running the code in a Jupyter notebook:
#Page()

In [3]:
if FA_database.value == "Scheveningen":
    # fa_database = Path(r"/mnt/c/Users/santjer/OneDrive - Stichting Deltares/Documents/DestinE/Technical/01_FloodAdapt/scheveningen")
    fa_database = Path(r"C:\Users\santjer\OneDrive - Stichting Deltares\Documents\DestinE\Technical\01_FloodAdapt\scheveningen")
elif FA_database.value == "Option B":
    fa_database = Path(r"/mnt/c/Users/santjer/OneDrive - Stichting Deltares/Documents/DestinE/Technical/01_FloodAdapt/OPTIONB/") #########################
else:
    fa_database = Path(r"/mnt/c/Users/santjer/OneDrive - Stichting Deltares/Documents/DestinE/Technical/01_FloodAdapt/OPTIONC/") #########################


# HERE: whatever the solution is; people won't have datasets on their local drive

# Path to data folder. If (relative) file paths are used when specifying data sets, it will be looked for in this folder.
# data_folder = Path("/home/wotromp/InterTwin/Data")  


# OPTIONAL: HydroMT data catalogue from which to fetch data. The order matters here, as datasets are looked for in a catalogue in order from first catalogue name in list to last.
data_catalogues = [Path(r"/mnt/c/Users/santjer/OneDrive - Stichting Deltares/Documents/DestinE/Technical/01_FloodAdapt/deltares_data_wsl.yml")]

# Demonstrator

In [4]:
# >>>>>>>>>>>>>>>> FUNCTIONS ONLY <<<<<<<<<<<<<<<<<<<<<

############################################## GENERAL ############################################

def handle_draw(target, action, geo_json):
    print(f"Action: {action}, GeoJSON: {geo_json}") 
    if action == 'created' and 'geometry' in geo_json:
        geo_type = geo_json['geometry']['type']
        coords = geo_json['geometry']['coordinates']
        
        if current_geometry_type.value and current_geometry_type.value != geo_type:
            error_message.value = f"**Error**: You cannot draw a {geo_type} after a {current_geometry_type.value}. Please clear the previous geometry before adding a new type."
            return
        
        if geo_type == 'Polygon':
            polygon_coords = coords[0]
            selected_geometry.value = {'type': 'Polygon', 'coordinates': polygon_coords}
            drawn_polygons.value.append({'type': 'Polygon', 'coordinates': polygon_coords})
            print("Polygon coordinates updated:", selected_geometry.value)
        
        elif geo_type == 'LineString':  # Handling polylines
            selected_geometry.value = {'type': 'LineString', 'coordinates': coords}
            drawn_polylines.value.append({'type': 'LineString', 'coordinates': coords})
            print("Polyline coordinates updated:", selected_geometry.value)

        elif geo_type == 'Point':  # Handling markers
            if point_placed.value:
                error_message.value = "**Error**: Only one point can be placed. Please first save the measure before adding another one."
                return
            selected_geometry.value = {'type': 'Point', 'coordinates': coords}
            drawn_markers.value.append({'type': 'Point', 'coordinates': coords})
            point_placed.set(True)  # Set point placed flag to True
            print("Marker coordinates updated:", selected_geometry.value)

        current_geometry_type.value = geo_type  # Update the current geometry type
        error_message.value = ""

def update_draw_tools_none():
    global draw_control
    m.remove_control(draw_control)
    new_draw_control = DrawControl(polyline={},polygon={},circlemarker={})
    draw_control = new_draw_control
    draw_control.on_draw(handle_draw)
    m.add_control(draw_control)

############################################## TAB 1 ##############################################

def save_inputs(event_name, start_date, end_date):
    global event_dict
    event_dict["name"] = event_name
    event_dict["start_time"] = start_date.strftime('%Y-%m-%d %H:%M:%S')
    event_dict["end_time"] = end_date.strftime('%Y-%m-%d %H:%M:%S')
    event_dict["data_catalogues"] = 'MISSING'
    event_dict["sfincs_forcing"] = {'meteo': 'MISSING', 'waterlevel': 'MISSING'}

############################################## TAB 2 ##############################################

def save_inputs_projections():
        global proj_dict
        proj_dict["name"] = projName.value
        proj_dict["physical_projection"] = {'sea_level_rise': SLR_input.value,
                                            'rainfall_increase': RAIN_input.value}
        proj_dict["socio_economic_change"] = {'population_growth_existing': POP_input.value,
                                            'economic_growth': ECON_input.value}

############################################## TAB 3 ##############################################

def clear_all_drawn_geometries():
    global draw_control
    m.remove_control(draw_control)
    draw_control = DrawControl(
        polyline={'shapeOptions': {'color': 'blue', 'weight': 4}},
        polygon={'shapeOptions': {'color': 'red', 'weight': 4}},
        marker={},
        rectangle={},
        circle={},    )
    draw_control.on_draw(handle_draw)
    m.add_control(draw_control)
    drawn_polylines.set([])
    drawn_polygons.set([])
    drawn_markers.set([])
    point_placed.set(False) 

def update_draw_tools():
    print(f"Updating draw tools for measure_type: {measure_type.value}")
    global draw_control
    m.remove_control(draw_control)
    
    if measure_type.value == "Floodwall":
        new_draw_control = DrawControl(polyline={'shapeOptions': {'color': 'blue', 'weight': 4}},polygon={},circlemarker={})
    elif measure_type.value == "Pump":
        new_draw_control = DrawControl(polyline={},polygon={},marker={})  # Enable marker only
    elif measure_type.value in ["Water square", "Green infrastructure", "Water storage", "Elevate properties", "Floodproof properties", "Buyout properties"]:
        new_draw_control = DrawControl(polyline={},polygon={'shapeOptions': {'color': 'red', 'weight': 4}},circlemarker={})

    draw_control = new_draw_control
    draw_control.on_draw(handle_draw)
    m.add_control(draw_control)

    drawn_polylines.set([])
    drawn_polygons.set([])
    drawn_markers.set([])
    point_placed.set(False)

def save_current_add_new():
        if selected_geometry.value is None:
            error_message.value = "**Error**: Please place the measure on the map."
            return
        else:
            error_message.set("")
        
        measure_misc  = {}
        if measure_type.value == "Floodwall":
            mtype = "floodwall"
            measure_misc = {"elevation": {"value": elevation_fw.value, "units": 'meters'}}
        elif measure_type.value == "Pump":
            mtype = "pump"
            measure_misc = {"discharge": {"value": discharge_fw.value, "units": 'm3/s'}}
        elif measure_type.value == "Water square":
            mtype = "water_square"
            measure_misc = {"volume": {"value": volume_ws.value, "units": 'm3'},
                            "height": {"value": height_ws.value, "units": "meters"}}
        elif measure_type.value == "Green infrastructure":
            mtype = "greening"
            measure_misc = {"volume": {"value": volume_gi.value, "units": 'm3'},
                            "height": {"value": height_gi.value, "units": "meters"},
                            "percent_area": percarea_gi.value}
        elif measure_type.value == "Water storage":
            mtype = "total_storage"
            measure_misc = {"volume": {"value": volume_wst.value, "units": 'm3'}}
        elif measure_type.value == "Elevate properties":
            mtype = "elevate_properties"
            measure_misc = {"elevation": {"value": elevation_prop.value, "units": 'meters'}}
        elif measure_type.value == "Floodproof properties":
            mtype = "floodproof_properties"
            measure_misc = {"elevation": {"value": elevation_fpprop.value, "units": 'meters'}}
        elif measure_type.value == "Buyout properties":
            mtype = "buyout_properties"
            measure_misc = {}

        measure = {
            "name": measureName.value,
            "type": mtype, # measure_type.value,
            "selection": selected_geometry.value,
            "misc": measure_misc        }
        measures_list.value.append(measure)
        save_directory = Path.cwd() # Path(r"/mnt/c/Users/santjer/OneDrive - Stichting Deltares/Documents/DestinE/Technical/02_Solara")
        geojson_data = {"measure_area": selected_geometry.value}
        geojson_filename = save_directory / f"{measureName.value}_geometry.geojson"
        with open(geojson_filename, 'w') as geojson_file:
            json.dump(geojson_data, geojson_file)
        measure["selection"] = str(geojson_filename) # - - - - - - - - - - - - - - - - - - - - - - - -
        clear_all_drawn_geometries()
        selected_geometry.set(None)
        current_geometry_type.set(None)
        measureName.set("Measure Name")
        measure_type.set("Floodwall")
        elevation_fw.set(0)
        discharge_fw.set(0) 
        volume_ws.set(0) 
        height_ws.set(0) 
        volume_gi.set(0) 
        height_gi.set(0) 
        percarea_gi.set(0)
        volume_wst.set(0) 
        elevation_prop.set(0) 
        elevation_fpprop.set(0)

def save_all_measures():
    global strategy
    error_message.set("")
    clear_all_drawn_geometries()
    selected_geometry.set(None)
    current_geometry_type.set(None)
    
    strategy = {'name': strategyName.value}
    for idx, measure in enumerate(measures_list.value, start=1):
        if measure['selection'] is None:
            continue
        measure_name = f"measure{idx}"
        file_path = Path.cwd() / f"{measure['name']}_geometry.geojson"# f"{measureName.value}_geometry.geojson" #####################
        strategy[measure_name] = {
            'name': measure['name'],
            'type': measure['type'].lower(), 
            'selection': str(file_path), # str(geojson_filename),
            'misc': measure['misc']}
    strategyName.set("Strategy Name")

def clear_measures():
    global strategy
    for measure in measures_list.value:
        measure_name = measure['name']
        file_path = Path.cwd() / f"{measure_name}_geometry.geojson"
        if file_path.exists():
            file_path.unlink()
    measures_list.set([]) 
    selected_geometry.set(None)
    current_geometry_type.set(None)
    error_message.set("")
    clear_all_drawn_geometries()
    measureName.set("Measure Name")
    measure_type.set("Floodwall")
    strategyName.set("Strategy Name")
    strategy = {} 

def save_strategy():
    save_current_add_new()
    save_all_measures()

############################################## TAB 4 ##############################################
def initialise_scenario():
    global scenario
    scenario = {
        'name': scenarioName.value, # "test_scenario",
        'event': event_dict,
        'projection': proj_dict,
        'strategy': strategy,
    }

def save_and_run():
    initialise_scenario()

    scenario_fn = fa_database / f"{scenario['name']}_toplevel.toml"
    with open(scenario_fn, 'wb+') as f:
            tomli_w.dump(scenario, f)

    database_fn = Path.cwd()
    scenario_name = scenarioName.value
    create_workflow_config(database_fn, scenario_name)     # NOT SUCCESSFUL!!! ###############################################################
    run_fa_scenario_workflow(database_fn, scenario_name)   # NOT SUCCESSFUL!!! ###############################################################

############################################## TAB 5  ##############################################

def on_checkbox_change(checkbox_id):
    if checkbox_id == 1:
        mapchoice1.set(True)
        mapchoice2.set(False)
        mapchoice3.set(False)
        selected_view.set("map1")
    elif checkbox_id == 2:
        mapchoice1.set(False)
        mapchoice2.set(True)
        mapchoice3.set(False)
        selected_view.set("map2")
    elif checkbox_id == 3:
        mapchoice1.set(False)
        mapchoice2.set(False)
        mapchoice3.set(True)
        selected_view.set("map3")
    else:
        selected_view.set(None)

database_path = Path(r"C:\Users\santjer\OneDrive - Stichting Deltares\Documents\DestinE\Technical\01_FloodAdapt\scheveningen_visualisation\scheveningen") # Path("/home/wotromp/DE372/scheveningen")
scenario = "test_scenario"


floodmap_image_path = Path("flood_map.png")
show_waiting = solara.reactive(False)
def generate_floodmap():
    data_catalog = ['deltares_data']
    dc = DataCatalog(data_libs=data_catalog)
    floodmap_path = database_path/"output"/"Scenarios"/scenario/"Flooding"/"FloodMap_test_scenario.tif"
    sf_path = database_path/"output"/"Scenarios"/scenario/"Flooding"/"simulations"/"overland"
    sf = SfincsModel(root=sf_path, mode='r')
    floodmap = xr.open_dataarray(floodmap_path).rio.reproject(4326)
    region = sf.region.to_crs(4326)
    gswo = dc.get_rasterdataset("gswo", geom=region, buffer=100)
    gswo_mask = gswo.raster.reproject_like(floodmap, method="max") <= 5
    floodmap = floodmap.where(gswo_mask)

    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(projection=proj)
    floodmap.plot(ax=ax, vmin=0, vmax=3, cmap="Blues", cbar_kwargs={'shrink': 0.3, "pad": 0.02})
    ax.set_xlim(region.total_bounds[[0, 2]])
    ax.set_ylim(region.total_bounds[[1, 3]])
    ax.add_image(cimgt.OSM(), 12, interpolation='bilinear', alpha=0.5)
    ax.set_ylabel("Latitude [deg]")
    ax.set_xlabel("Longitude [deg]")
    ax.set_title("Maximum Flood Extent")
    
    plt.savefig(floodmap_image_path, format='png', bbox_inches='tight')
    plt.close(fig)

    show_waiting.set(False)

def generate_floodmap_in_thread():
    show_waiting.set(True)
    threading.Thread(target=generate_floodmap).start() 


fiatoutput_image_path = Path("fiat_output.png")
show_waiting = solara.reactive(False)
def generate_fiat_output():
    fiat_path = database_path/"output"/"Scenarios"/scenario/"Impacts"/"fiat_model"
    fiat = FiatModel(root=fiat_path, mode='r')
    fiat.read()
    gdf_fiat = fiat.exposure.get_full_gdf(fiat.exposure.exposure_db)
    fiat_output_path = fiat_path/"output"/"spatial.gpkg"
    fiat_output = gpd.read_file(fiat_output_path).to_crs("EPSG:4326")

    # THIS COLUMN DOESNT EXIST! gdf_fiat["Max Potential Damage: Total"] ####################################################################################################################
    print('ERROR: THIS COLUMN DOESNT EXIST! gdf_fiat["Max Potential Damage: Total"]')
    print('thus assuming total = structure+content')
    fiat_output["Total Damage Normalized"] = fiat_output['Total Damage']/(gdf_fiat["Max Potential Damage: Structure"]+gdf_fiat["Max Potential Damage: Content"])

    fiat_output["Inundation Depth"].dropna() # fiat_output["Inundation Depth"].dropna().plot()

    interval = np.linspace(0.25,1)
    clrs = plt.cm.YlOrRd(interval)
    cmap = colors.LinearSegmentedColormap.from_list('name', clrs)

    fig = plt.figure(figsize=(14,10))
    ax = fig.add_subplot(projection=proj)
    ax.add_image(cimgt.OSM(),10, interpolation='bilinear', alpha=0.3)
    fiat_output.plot(ax=ax, column="Total Damage Normalized", legend=True, cmap=cmap,
        legend_kwds={'label': "Normalized Damages", "shrink": 0.4, "pad": 0.02})
    ax.set_title("Building Damages, Normalized")
    ax.xaxis.set_visible(True)
    ax.yaxis.set_visible(True)
    ax.set_ylabel("Latitude [deg]")
    ax.set_xlabel("Longitude [deg]")

    plt.savefig(fiatoutput_image_path, format='png', bbox_inches='tight')
    plt.close(fig)

    show_waiting.set(False)

def generate_fiat_output_in_thread():
    show_waiting.set(True)
    threading.Thread(target=generate_fiat_output).start() 


def map1():
    if not floodmap_image_path.exists():
        generate_floodmap_in_thread() # generate_floodmap()

def map2():
    if not fiatoutput_image_path.exists():
        generate_fiat_output_in_thread() # generate_fiat_output()

def map3():
    fig, ax = plt.subplots(figsize=(6, 4))
    ax.plot([3, 2, 1], [4, 5, 6])
    plt.title("Map 3 Plot")
    solara.FigureMatplotlib(fig)

###################################################################################################

In [ ]:
# >>>>>> CORRECT BEST VERSION OF CODE BELOW \_v_/ without functions (find them on top) <<<<<<<<<<<<<<<<<<<

polyline_coordinates = solara.reactive([])
polygon_coordinates = solara.reactive([])
marker_coordinates = solara.reactive([])
drawn_polylines = solara.reactive([])
drawn_polygons = solara.reactive([])
drawn_markers = solara.reactive([])
selected_geometry = solara.reactive(None) 
selected_tab = solara.reactive('Event') 
measures_list = solara.reactive([])
error_message = solara.reactive("")
current_geometry_type = solara.reactive(None)
point_placed = solara.reactive(False)
open_delete_confirmation = solara.reactive(False)

m = Map(center=(52.08654741528378, 4.295223531699989), zoom=10, scroll_wheel_zoom=True, basemap=basemaps.OpenStreetMap.Mapnik)

draw_control = DrawControl(
    polyline={'shapeOptions': {'color': 'blue', 'weight': 4}},
    polygon={'shapeOptions': {'color': 'red', 'weight': 4}},
    marker={},
    rectangle={},  
    circle={},
)

draw_control.on_draw(handle_draw)
m.add_control(draw_control)

map_output = Output()
with map_output:
    display(m)

####################################### TAB 1 WEATHER EVENT #######################################
event_dict = {}

@solara.component
def TabEvent():
    update_draw_tools_none()
    start_date = solara.use_reactive(dt.date.today())
    end_date = solara.use_reactive(dt.date.today())
    eventName = solara.use_reactive("Event Name")
    
    with solara.Card("Configure the Weather Event", style={"width": "100%", "padding": "10px"}):
        solara.InputText("Event Name", value=eventName, continuous_update=True)
        solara.Markdown(f"**Your Event Name**: {eventName.value}")

        solara.Text("Select the Start Date:")
        solara.lab.InputDate(start_date)
        
        solara.Text("Select the End Date:")
        solara.lab.InputDate(end_date)
        
        if end_date.value < start_date.value:
            solara.Markdown("**Warning**: The end date cannot be earlier than the start date.", style={"color": "red"})
        
        solara.Button("Save Inputs", on_click=lambda: save_inputs(eventName.value, start_date.value, end_date.value))


####################################### TAB 2 PROJECTIONS #######################################
SLR_input = solara.reactive(0.0)
RAIN_input = solara.reactive(0.0)
POP_input = solara.reactive(0.0)
ECON_input = solara.reactive(0.0)
projName = solara.reactive("Projection Name")
proj_dict = {}

@solara.component
def TabProjections():
    update_draw_tools_none()

    with solara.Card("Projections", style={"width": "100%", "padding": "10px"}):
        solara.InputText("Projection Name", value=projName, continuous_update=True)
        solara.Markdown(f"**Your Projection Name**: {projName.value}")
        solara.InputFloat("Sea Level Rise (metres)", value=SLR_input, continuous_update=True)
        solara.InputFloat("Precipitation Increase (%)", value=RAIN_input, continuous_update=True)
        solara.InputFloat("Population Growth (%)", value=POP_input, continuous_update=True)
        solara.InputFloat("Economic Growth (%)", value=ECON_input, continuous_update=True)
        
        solara.Button("Save Inputs", on_click=save_inputs_projections, style={"margin-top": "20px"})

####################################### TAB 3 MEASURES #######################################
strategyName = solara.reactive("Strategy Name")
measureName = solara.reactive("Measure Name")
measure_types = ["Floodwall","Pump","Water square","Green infrastructure","Water storage",
                 "Elevate properties","Floodproof properties","Buyout properties"]
measure_type = solara.reactive("Floodwall")
elevation_fw = solara.reactive(0)  # For floodwall 
discharge_fw = solara.reactive(0)  # For pump 
volume_ws = solara.reactive(0)  # For water square
height_ws = solara.reactive(0)  # For water square 
volume_gi = solara.reactive(0)  # For green infrastructure
height_gi = solara.reactive(0)  # For green infrastructure
percarea_gi = solara.reactive(0)  # For green infrastructure 
volume_wst = solara.reactive(0)  # For water storage 
elevation_prop = solara.reactive(0)  # For properties 
elevation_fpprop = solara.reactive(0)  # For floodproof properties 
strategy = {} 

@solara.component
def TabMeasures(): 
    with solara.Card("Strategy Name", style={"width": "100%", "padding": "10px"}):
        solara.InputText("Strategy Name", value=strategyName, continuous_update=True)
        
    with solara.Card("Measures", style={"width": "100%", "padding": "10px"}):
        solara.InputText("Measure Name", value=measureName, continuous_update=True)
        solara.Markdown(f"**Your Measure Name**: {measureName.value}")
        
        duplicate_found = any(measure["name"] == measureName.value for measure in measures_list.value) 
        if duplicate_found:
            solara.Markdown(f"**Error**: A measure with the name '{measureName.value}' already exists. Please choose a different name.", style={"color": "red"})
        
        solara.Select(label="Measure Type", value=measure_type, values=measure_types)

        if measure_type.value == "Floodwall":
            solara.InputFloat("Elevation [m]", value=elevation_fw, continuous_update=True)
            solara.use_effect(lambda: update_draw_tools(), [measure_type.value])
        elif measure_type.value == "Pump":
            solara.InputFloat("Discharge [m³/s]", value=discharge_fw, continuous_update=True)
            solara.use_effect(lambda: update_draw_tools(), [measure_type.value])
        elif measure_type.value == "Water square":
            solara.InputFloat("Volume [m³]", value=volume_ws, continuous_update=True)
            solara.InputFloat("Height [m]", value=height_ws, continuous_update=True)
            solara.use_effect(lambda: update_draw_tools(), [measure_type.value])
        elif measure_type.value == "Green infrastructure":
            solara.InputFloat("Volume [m³]", value=volume_gi, continuous_update=True)
            solara.InputFloat("Height [m]", value=height_gi, continuous_update=True)
            solara.InputFloat("Percentage of area [%]", value=percarea_gi, continuous_update=True)
            solara.use_effect(lambda: update_draw_tools(), [measure_type.value])
        elif measure_type.value == "Water storage":
            solara.InputFloat("Volume [m³]", value=volume_wst, continuous_update=True)
            solara.use_effect(lambda: update_draw_tools(), [measure_type.value])
        elif measure_type.value == "Elevate properties":
            solara.InputFloat("Elevation [m]", value=elevation_prop, continuous_update=True)
            solara.use_effect(lambda: update_draw_tools(), [measure_type.value])
        elif measure_type.value == "Floodproof properties":
            solara.InputFloat("Elevation [m]", value=elevation_fpprop, continuous_update=True)
            solara.use_effect(lambda: update_draw_tools(), [measure_type.value])
        elif measure_type.value == "Buyout properties":
            solara.use_effect(lambda: update_draw_tools(), [measure_type.value])

        solara.Markdown("Click on the map to place the measure location (or draw a shape).")
        if measures_list.value:
            selected_measures = [f"{measure['type']}, '{measure['name']}'" for measure in measures_list.value]
            formatted_measures = "\n".join(selected_measures)
            solara.Markdown(f"**Selected Measures:**\n{formatted_measures}")

        if error_message.value:
            solara.Markdown(error_message.value, style={"color": "red"})

        with solara.Row():
            solara.Button("Save Current and Add New Measure", on_click=save_current_add_new, style={"margin-top": "20px", "margin-left": "-20px"})
            solara.Button("Clear All", on_click=clear_measures, style={"margin-top": "80px", "margin-left": "-348px"})
            solara.Button("Save Strategy", on_click=save_strategy, style={"margin-top": "80px", "margin-left": "54px"})


####################################### TAB 4 SAVE #######################################
scenarioName = solara.reactive("Scenario Name")

@solara.component
def TabSaveRun():
    update_draw_tools_none() 

    with solara.Card("Save inputs and Run model", style={"width": "100%", "padding": "10px"}):
        solara.InputText("Scenario Name", value=scenarioName, continuous_update=True)
        solara.Markdown(f"**Your Scenario Name**: {scenarioName.value}")
        solara.Markdown(f"<span style='color: #FF4500;'><b>Note to be removed later:</b> save_and_run not successful: error in two functions </span>")

        solara.Button("Save & Run", on_click=lambda: save_and_run()) 

####################################### TAB 5 MAP VISUALISATION #######################################
mapchoice1 = solara.reactive(False)
mapchoice2 = solara.reactive(False)
mapchoice3 = solara.reactive(False)
selected_view = solara.reactive(None)  

@solara.component
def TabMapVisualisation():
    update_draw_tools_none()
    with solara.Card("Map Visualisation", style={"width": "100%", "padding": "10px"}):
        solara.Markdown(f"<span style='color: #4682B4;'><b>Please Note:</b> The initial generation of figures may take some time to complete.</span>")
        solara.Markdown(f"<span style='color: #FF4500;'><b>Note to be removed later:</b> if floodmap doesn't work: open directory in e.g. file explorer: P:\wflow_global\hydromt\hydrography\gswo </span>")
        solara.Markdown(f"**Selection**:")
        solara.Checkbox(label="Floodmap", value=mapchoice1, on_value=lambda v: on_checkbox_change(1) if v else None)
        solara.Checkbox(label="Building Damages", value=mapchoice2, on_value=lambda v: on_checkbox_change(2) if v else None)
        solara.Checkbox(label="Dummy plot", value=mapchoice3, on_value=lambda v: on_checkbox_change(3) if v else None)


###########################################################################################
###########################################################################################
###########################################################################################
###########################################################################################
###########################################################################################
@solara.component
def SettingsTabs():
    with solara.Row(gap="0px"):
        with solara.Columns():
            solara.Button("Event", on_click=lambda: (selected_tab.set('Event'), selected_view.set(None)))
            solara.Button("Projections", on_click=lambda: (selected_tab.set('Projections'), selected_view.set(None)))
            solara.Button("Measures", on_click=lambda: (selected_tab.set('Measures'), selected_view.set(None)))
    with solara.Row(gap="0px"):
        with solara.Columns():
            solara.Button("Save & Run", on_click=lambda: (selected_tab.set('TabSaveRun'), selected_view.set(None)))
            solara.Button("Visualise Maps", on_click=lambda: selected_tab.set('TabMapVisualisation'))
 
    if selected_tab.value == 'Event':
        TabEvent()
    elif selected_tab.value == 'Projections':
        TabProjections()
    elif selected_tab.value == 'Measures':
        TabMeasures()
    elif selected_tab.value == 'TabSaveRun':
        TabSaveRun()
    elif selected_tab.value == 'TabMapVisualisation':
        TabMapVisualisation()
        mapchoice1.set(False); mapchoice2.set(False); mapchoice3.set(False)

@solara.component
def App():
    with solara.Columns():
        with solara.Column(style={"width": "70%", "min-width": "650px"}):
            if show_waiting.value:
                solara.ProgressLinear(True) # solara.SpinnerSolara(size="100px") # spinner or progressbar?
            else:
                if selected_view.value == "map1":
                    map1()
                    solara.Image(str(floodmap_image_path))
                elif selected_view.value == "map2":
                    map2()
                    solara.Image(str(fiatoutput_image_path))
                elif selected_view.value == "map3":
                    map3()
                else:
                    display(m) 

        with solara.Column(style={"width": "30%", "min-width": "500px"}):
            SettingsTabs()

App()

# Interactive FloodMap

## static floodmap plot

old:

In [ ]:
%matplotlib inline
plt.close('all')

database_path = Path(r"C:\Users\santjer\OneDrive - Stichting Deltares\Documents\DestinE\Technical\01_FloodAdapt\scheveningen_visualisation\scheveningen") # Path("/home/wotromp/DE372/scheveningen")
scenario = "test_scenario"


floodmap_image_path = Path("flood_map.png")
show_waiting = solara.reactive(False)

data_catalog = ['deltares_data']
dc = DataCatalog(data_libs=data_catalog)
floodmap_path = database_path/"output"/"Scenarios"/scenario/"Flooding"/"FloodMap_test_scenario.tif"
sf_path = database_path/"output"/"Scenarios"/scenario/"Flooding"/"simulations"/"overland"
sf = SfincsModel(root=sf_path, mode='r')
floodmap = xr.open_dataarray(floodmap_path).rio.reproject(4326)
region = sf.region.to_crs(4326)
gswo = dc.get_rasterdataset("gswo", geom=region, buffer=100)
gswo_mask = gswo.raster.reproject_like(floodmap, method="max") <= 5
floodmap = floodmap.where(gswo_mask)

fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(projection=proj)
floodmap.plot(ax=ax, vmin=0, vmax=3, cmap="Blues", cbar_kwargs={'shrink': 0.3, "pad": 0.02})
ax.set_xlim(region.total_bounds[[0, 2]])
ax.set_ylim(region.total_bounds[[1, 3]])
ax.add_image(cimgt.OSM(), 12, interpolation='bilinear', alpha=0.5)
ax.set_ylabel("Latitude [deg]")
ax.set_xlabel("Longitude [deg]")
ax.set_title("Maximum Flood Extent")

plt.show()

easy:

In [ ]:
import matplotlib.pyplot as plt
import xarray as xr
from pathlib import Path


database_path = Path(r"C:\Users\santjer\OneDrive - Stichting Deltares\Documents\DestinE\Technical\01_FloodAdapt\scheveningen_visualisation\scheveningen")
scenario = "test_scenario"
floodmap_path = database_path / "output" / "Scenarios" / scenario / "Flooding" / "FloodMap_test_scenario.tif"

floodmap = xr.open_dataarray(floodmap_path).rio.reproject(4326)

flood_data = floodmap.values[0]  # remove band dimension if it's just one

x_values = floodmap.coords['x'].values
y_values = floodmap.coords['y'].values

vmin = 0 
vmax = 0.01

plt.figure(figsize=(10, 6))
im = plt.imshow(flood_data, extent=[x_values.min(), x_values.max(), y_values.min(), y_values.max()], origin='upper', cmap='Blues', vmin=vmin, vmax=vmax)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Floodmap')
cbar = plt.colorbar(im,label='Flood Level')
cbar.mappable.set_clim(vmin=vmin, vmax=vmax) 

plt.show()


## flexible maps, 2 versions

black everywhere

In [ ]:
from ipyleaflet import Map, basemaps, TileLayer, LegendControl
from localtileserver import TileClient
import xarray as xr
import rioxarray
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
import os

if os.path.exists("temp_floodmap.tiff"):
    os.remove("temp_floodmap.tiff")

vmin = 0
vmax = 21

database_path = Path(r"C:\Users\santjer\OneDrive - Stichting Deltares\Documents\DestinE\Technical\01_FloodAdapt\scheveningen_visualisation\scheveningen")
scenario = "test_scenario"
floodmap_path = database_path / "output" / "Scenarios" / scenario / "Flooding" / "FloodMap_test_scenario.tif"

floodmap = xr.open_dataarray(floodmap_path).rio.reproject(4326)
floodmap_clipped = floodmap.clip(min=vmin, max=vmax)

mask = (floodmap_clipped <= 0) | np.isnan(floodmap_clipped) # mask = (floodmap_clipped <= 0) # mask = (floodmap_clipped <= 0) | np.isnan(floodmap_clipped)
floodmap_clipped = floodmap_clipped.where(~mask, np.nan)

cmap = plt.get_cmap('Blues') 
colors = cmap(np.linspace(0, 1, cmap.N))
colors[:, -1] = 0 
custom_cmap = ListedColormap(colors)

# INCLUDING THIS LINE BELOW MAKES DOTS APPEAR! NOT INCLUDING IT MAKES EVERYTHIGN BLACK!
# floodmap_clipped = floodmap_clipped.rio.write_nodata(np.nan) # floodmap_clipped.rio.write_nodata(0) # floodmap_clipped = 
floodmap_clipped.rio.to_raster("temp_floodmap.tiff", dtype='float32')

tile_client = TileClient("temp_floodmap.tiff")

m = Map(center=(52.1, 4.3), zoom=10, basemap=basemaps.OpenStreetMap.Mapnik)

tile_layer = TileLayer(url=tile_client.get_tile_url(), opacity=1.0)
m.add_layer(tile_layer)


m

black dots

activating the indicated row (#--------) might change the appearance?

In [ ]:
from ipyleaflet import Map, basemaps, TileLayer
from localtileserver import TileClient
import xarray as xr
import rioxarray
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
import os

if os.path.exists("temp_floodmap.tiff"):
    os.remove("temp_floodmap.tiff")

vmin = 0
vmax = 21

database_path = Path(r"C:\Users\santjer\OneDrive - Stichting Deltares\Documents\DestinE\Technical\01_FloodAdapt\scheveningen_visualisation\scheveningen")
scenario = "test_scenario"
floodmap_path = database_path / "output" / "Scenarios" / scenario / "Flooding" / "FloodMap_test_scenario.tif"

floodmap = xr.open_dataarray(floodmap_path).rio.reproject(4326)
floodmap_clipped = floodmap.clip(min=vmin, max=vmax)

mask = (floodmap_clipped <= 0) | np.isnan(floodmap_clipped)

# floodmap_clipped = floodmap_clipped.where(~mask, np.nan) # ------------------------------------------------------------------------------------------

cmap = plt.get_cmap('Blues')

colors = cmap(np.linspace(0, 1, cmap.N))

colors[:, -1] = 1  
colors[0, -1] = 0  
custom_cmap = ListedColormap(colors)

bounds = np.linspace(vmin, vmax, (vmax - vmin) + 1)
norm = BoundaryNorm(bounds, custom_cmap.N)

floodmap_clipped.rio.set_nodata(np.nan)

floodmap_clipped.rio.to_raster("temp_floodmap.tiff", dtype='float32')

tile_client = TileClient("temp_floodmap.tiff")

m = Map(center=(52.1, 4.3), zoom=10, basemap=basemaps.OpenStreetMap.Mapnik)

tile_layer = TileLayer(url=tile_client.get_tile_url(), opacity=1.0)
m.add_layer(tile_layer)


m



# FIAT map

load data and import packages

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import xarray as xr
import rioxarray
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.cm import get_cmap
import matplotlib.patches as mpatches
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
import geopandas as gpd

from hydromt.data_catalog import DataCatalog
from hydromt_sfincs import SfincsModel
from hydromt_wflow import WflowModel
from hydromt_fiat.fiat import FiatModel
from hydromt_sfincs.utils import downscale_floodmap

from flood_adapt.api import static
from flood_adapt.api.scenarios import get_scenario
from DT_flood.utils.plot_utils import plot_wflow_model, plot_fiat_model, plot_sfincs_model

proj = ccrs.PlateCarree()



database_path = Path(r"C:\Users\santjer\OneDrive - Stichting Deltares\Documents\DestinE\Technical\01_FloodAdapt\scheveningen_visualisation\scheveningen") # Path("/home/wotromp/DE372/scheveningen")
scenario = "test_scenario"

fiat_path = database_path/"output"/"Scenarios"/scenario/"Impacts"/"fiat_model"


fiat = FiatModel(root=fiat_path, mode='r')
fiat.read()
gdf_fiat = fiat.exposure.get_full_gdf(fiat.exposure.exposure_db)


fiat_output_path = fiat_path/"output"/"spatial.gpkg"


fiat_output = gpd.read_file(fiat_output_path).to_crs("EPSG:4326")


# THIS COLUMN DOESNT EXIST! gdf_fiat["Max Potential Damage: Total"]
print('ERROR: THIS COLUMN DOESNT EXIST! gdf_fiat["Max Potential Damage: Total"]')
print('thus assuming total = structure+content')
fiat_output["Total Damage Normalized"] = fiat_output['Total Damage']/(gdf_fiat["Max Potential Damage: Structure"]+gdf_fiat["Max Potential Damage: Content"])


fiat_output["Inundation Depth"].dropna() # fiat_output["Inundation Depth"].dropna().plot()



INFO:hydromt_fiat.fiat:Initializing fiat model from hydromt_fiat (v0.3.2).
INFO:hydromt_fiat.fiat:Reading model data from C:\Users\santjer\OneDrive - Stichting Deltares\Documents\DestinE\Technical\01_FloodAdapt\scheveningen_visualisation\scheveningen\output\Scenarios\test_scenario\Impacts\fiat_model
INFO:hydromt_fiat.fiat:Reading model table files.
INFO:hydromt_fiat.fiat:Reading exposure geometries.
INFO:hydromt_fiat.fiat:Setting geometry name to buildings...
INFO:hydromt_fiat.fiat:Setting exposure geometries...
INFO:hydromt_fiat.fiat:Setting geometry name to roads...
INFO:hydromt_fiat.fiat:Setting exposure geometries...


ERROR: THIS COLUMN DOESNT EXIST! gdf_fiat["Max Potential Damage: Total"]
thus assuming total = structure+content


4       -0.20
44      -0.19
75      -0.18
79      -0.19
120     -0.20
         ... 
21560    0.96
21571   -0.19
21574   -0.20
21585   -0.19
21609   -0.18
Name: Inundation Depth, Length: 552, dtype: float64

## static

In [ ]:
# STATIC PLOT

# fiat_output.plot(column="Total Damage Normalized", cmap='plasma')

interval = np.linspace(0.25,1)
clrs = plt.cm.YlOrRd(interval)
cmap = colors.LinearSegmentedColormap.from_list('name', clrs)


fig = plt.figure(figsize=(14,10))
ax = fig.add_subplot(projection=proj)
ax.add_image(cimgt.OSM(),10, interpolation='bilinear', alpha=0.3)
fiat_output.plot(ax=ax, column="Total Damage Normalized", legend=True, cmap=cmap,
    legend_kwds={'label': "Normalized Damages", "shrink": 0.4, "pad": 0.02})

ax.set_title("Building Damages, Normalized")
ax.xaxis.set_visible(True)
ax.yaxis.set_visible(True)
ax.set_ylabel("Latitude [deg]")
ax.set_xlabel("Longitude [deg]")

fiatoutput_image_path = Path("fiat_output.png")

plt.show()

# plt.savefig(fiatoutput_image_path,format='png',bbox_inches='tight')
# plt.close(fig)

## fiat map into interactive map

In [ ]:
# more flexible but less successful plot
from ipyleaflet import Map, basemaps, TileLayer, LegendControl
from localtileserver import TileClient
import geopandas as gpd
import xarray as xr
import rioxarray
import numpy as np
from rasterio.features import rasterize
from shapely.geometry import mapping
import os
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors


temp_tiff_path = "temp_fiat_output.tiff"
if os.path.exists(temp_tiff_path):
    os.remove(temp_tiff_path)

minx, miny, maxx, maxy = 4.23, 52.07, 4.31, 52.122
pixel_size = 0.0001 # 0.00005 0.0005
width = int((maxx - minx) / pixel_size)
height = int((maxy - miny) / pixel_size)

fiat_raster = xr.DataArray(
    np.full((height, width), np.nan), dims=("y", "x"),
    coords={"y": np.linspace(maxy, miny, height), "x": np.linspace(minx, maxx, width)},
)

shapes = ((mapping(geom), value) for geom, value in zip(fiat_output.geometry, fiat_output['Total Damage']))
fiat_raster.values = rasterize(
    shapes, out_shape=(height, width), transform=fiat_raster.rio.transform(), fill=np.nan, dtype='float32' # all_touched=True # dtype='float32' --statt alltouched-------------------
)

min_value = np.nanmin(fiat_raster.values)
max_value = np.nanmax(fiat_raster.values)

norm = mcolors.Normalize(vmin=min_value, vmax=max_value)
cmap = plt.cm.autumn_r # cividis # viridis

colored_image = (cmap(norm(fiat_raster.values))[:, :, :3] * 255).astype(np.uint8)
alpha_channel = np.ones(colored_image.shape[:2], dtype=np.uint8) * 255
alpha_channel[np.isnan(fiat_raster.values)] = 0

colored_image_rgba = np.dstack((colored_image, alpha_channel))

colored_image_raster = xr.DataArray(
    colored_image_rgba,
    dims=("y", "x", "color"),
    coords={"y": fiat_raster.y, "x": fiat_raster.x}
)

colored_image_raster = colored_image_raster.transpose("color", "y", "x")

colored_image_raster.rio.write_crs("EPSG:4326", inplace=True)
colored_image_raster.rio.to_raster(temp_tiff_path, driver='GTiff', dtype='uint8')

tile_client = TileClient(temp_tiff_path)
m = Map(center=(52.1, 4.3), zoom=10, basemap=basemaps.OpenStreetMap.Mapnik)

tile_layer = TileLayer(url=tile_client.get_tile_url(), opacity=0.5, name="Total Damage")
m.add_layer(tile_layer)


m


### SUCCESSFUL 1-liner

In [ ]:
fiat_output.explore(column="Total Damage Normalized")

# Metrics play around

In [ ]:
gdf_fiat

In [ ]:
fiat_output

- keep stuff which is in both
- double check nan values for damage, what happens?
- binning pychart



In [7]:
fiat_full = gdf_fiat.sjoin(fiat_output, how='inner', predicate='intersects') # joins those where there are matches in both GeoDataFrame (PREVIOUSLY 'left')
# fiat_full

,Object ID_left,geometry,Primary Object Type,Secondary Object Type,Max Potential Damage: Structure,Max Potential Damage: Content,Ground Floor Height,Extraction Method,Ground Elevation,Damage Function: Structure,...,Aggregation Label: Areas,SVI,index_right,Object ID_right,Inundation Depth,Reduction Factor,Damage: Structure,Damage: Content,Total Damage,Total Damage Normalized
40,544826386,POINT (4.26896 52.08959),residential,residential,683.504342,341.752171,0.2,centroid,5.497293,residential_structure,...,east_side,NaN,12,544826386,NaN,NaN,NaN,NaN,0.0,0.0
42,267699214,POINT (4.26159 52.09154),residential,residential,694.379515,347.189758,0.2,centroid,7.300887,residential_structure,...,west_side,NaN,14,267699214,NaN,NaN,NaN,NaN,0.0,0.0
43,544825880,POINT (4.27604 52.09000),residential,residential,716.133734,358.066867,0.2,centroid,4.506051,residential_structure,...,east_side,NaN,15,544825880,NaN,NaN,NaN,NaN,0.0,0.0
85,544826396,POINT (4.28170 52.09651),residential,residential,874.987971,437.493986,0.2,centroid,6.375336,residential_structure,...,east_side,NaN,29,544826396,NaN,NaN,NaN,NaN,0.0,0.0
86,544827034,POINT (4.27264 52.09558),residential,residential,905.773640,452.886820,0.2,centroid,7.099133,residential_structure,...,east_side,NaN,30,544827034,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24610,914,"POLYGON ((4.26277 52.09127, 4.26281 52.09124, ...",roads,residential,1.000000,NaN,0.0,NaN,7.490721,roads,...,NaN,NaN,1012,267699977,NaN,NaN,NaN,NaN,0.0,0.0
24613,1025,"POLYGON ((4.26287 52.09132, 4.26291 52.09128, ...",roads,residential,1.000000,NaN,0.0,NaN,7.490721,roads,...,NaN,NaN,14414,267703309,NaN,NaN,NaN,NaN,0.0,0.0
24621,1023,"POLYGON ((4.26364 52.09168, 4.26368 52.09165, ...",roads,residential,1.000000,NaN,0.0,NaN,7.380504,roads,...,NaN,NaN,14464,267703199,NaN,NaN,NaN,NaN,0.0,0.0
25341,909,"POLYGON ((4.27752 52.10612, 4.27755 52.10613, ...",roads,residential,1.000000,NaN,0.0,NaN,3.411886,roads,...,NaN,NaN,21176,250670639,NaN,NaN,NaN,NaN,0.0,0.0


filter to those where 'residential' is primary object type

In [ ]:
fiat_full[fiat_full['Primary Object Type'] == 'residential']

filter to those where 'total damage normalised' is (in fiat_output) >= 0.1, and cound the number

In [8]:
fiat_output.where(fiat_output["Total Damage Normalized"] >=0.1).count()

Object ID                  10
Inundation Depth           10
Reduction Factor           10
Damage: Structure          10
Damage: Content            10
Total Damage               10
geometry                   10
Total Damage Normalized    10
dtype: int64

In [17]:
damage_values = [0,0.1,0.5]
counts = []

for val in damage_values:
    counts.append(
        fiat_output.where(fiat_output["Total Damage Normalized"] >=val).count()['Total Damage Normalized']
    )

## create function for pie charts

In [26]:
def create_piechart(df, column, grouping, labels):
    counts = []
    for val in grouping:
        counts.append(
            df.where(df[column]>=val).count()[column]
        )
    
    df_counts = pd.DataFrame(
        data={"counts": counts},
        index=labels,
    )

    df_counts.plot.pie(y='counts')

In [ ]:
column = "Total Damage Normalized"
grouping = [0, 0.1, 0.5]
labels=[
    "No Damage",
    "Low Damage",
    "High Damage"
]

df = fiat_full[fiat_full['Primary Object Type'] == 'residential']

create_piechart(df, column, grouping, labels)

In [ ]:
counts

In [20]:
df = pd.DataFrame(
    data={"counts": counts},
    index=["Undamaged","Lightly Damaged", "Heavily Damages"]
)

In [ ]:
df.plot.pie(y='counts')

# Metrics clean